In [2]:
import regex as re
import pandas as pd

In [3]:
f=open('./openstack_10k.log')
lines=f.readlines()

In [4]:
split_pt=re.compile(r'([,\s$@<>\[\]\'"=]|\((?!\))|(?<!\()\)|[:\.](?=\s)|(?<=\b[a-zA-Z]+):|:(?=[a-zA-Z]+\b))')
hm_pt=re.compile(r'(^(?:\S+\s){6})(.*)')

In [5]:
header=[]
msg=[]
for line in lines:
    h,m=hm_pt.match(line).groups()
    
    header.append(h)
    msg.append(m)

In [6]:
print(header[0])
print(msg[0])

nova-api.log.1.2017-05-17_12:02:19 2017-05-16 16:26:12.995 25749 INFO nova.osapi_compute.wsgi.server 
[req-f706fe35-9183-45f5-b64e-bc242331859d 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1892 time: 0.2479329


In [7]:
header_tokens=[]
for h in header:
    tokens=[token for token in re.split(r'([\s$@<>\[\]\'"=]|\((?!\))|(?<!\()\)|[:\.,](?=\s)|(?<=\b[a-zA-Z]+):|:(?=[a-zA-Z]+\b))',h) if token!='' and token!=None]
    header_tokens.append(tokens)

In [8]:
msg_tokens=[]
for m in msg:
    tokens=[token for token in split_pt.split(m) if token!='' and token!=None]
    msg_tokens.append(tokens)

In [9]:
pd.DataFrame({'header_tokens':header_tokens,'msg_tokens':msg_tokens}).to_csv('tokens.csv')

In [10]:
header_anno=[]
for tokens in header_tokens:
    anno=['O']*len(tokens)
    anno[2],anno[4],anno[8],anno[10]='B-DATE','B-TIME','B-LVL','B-CLS'

    header_anno.append(anno)

In [11]:
list(zip(header_tokens[0],header_anno[0]))

[('nova-api.log.1.2017-05-17_12:02:19', 'O'),
 (' ', 'O'),
 ('2017-05-16', 'B-DATE'),
 (' ', 'O'),
 ('16:26:12.995', 'B-TIME'),
 (' ', 'O'),
 ('25749', 'O'),
 (' ', 'O'),
 ('INFO', 'B-LVL'),
 (' ', 'O'),
 ('nova.osapi_compute.wsgi.server', 'B-CLS'),
 (' ', 'O')]

In [12]:
msg_anno=[]
for tokens in msg_tokens:
    anno=['<O>']*len(tokens)
    msg_anno.append(anno)

In [13]:
re_date=r'\d{2}\/\d{2}\/\d{4}'
re_time=r'^\d{2}:\d{2}$'

re_obj=r'[a-zA-Z]+-\d{1,3}$'
re_cls=r'(\.([A-Z]+[a-z]*)+)|([A-Z]+[a-z]+){2,}$'
re_func=r'\S*\(\)$'
re_path=r'^\'?\/[a-zA-Z]'
re_ip=r'(?:[0-9]{1,3}\.){3}[0-9]{1,3}'

re_host=r':\d{3,5}$'
re_url=r'[a-z]+:\/\/\S*'


In [14]:
def sub_tag(ntokens,ntags, patt_tag_list):

    def _sub_tag(ntokens,ntags, patt, target_tag):
        for tokens,tags in zip(ntokens,ntags):
            for i in range(len(tokens)):
                if re.search(patt,tokens[i]):
                    tags[i]=target_tag

    for patt,tag in patt_tag_list:
        _sub_tag(ntokens,ntags,patt,tag)

In [15]:
sub_tag(msg_tokens,msg_anno,
    [(re_date,'B-DATE'),
    (re_time,'B-TIME'),
    (re_cls,'B-CLS'),
    (re_func,'B-FUNC'),
    (re_path,'B-PATH'),
    (re_ip,'B-IP'),
    (re_host,'B-HOST'),
    (re_url,'B-URL')])

In [16]:
list(zip(msg_tokens[0],msg_anno[0]))

[('[', '<O>'),
 ('req-f706fe35-9183-45f5-b64e-bc242331859d', '<O>'),
 (' ', '<O>'),
 ('113d3a99c3da401fbd62cc2caa5b96d2', '<O>'),
 (' ', '<O>'),
 ('54fadb412c4e40cdbaed9335e4c35a9e', '<O>'),
 (' ', '<O>'),
 ('-', '<O>'),
 (' ', '<O>'),
 ('-', '<O>'),
 (' ', '<O>'),
 ('-', '<O>'),
 (']', '<O>'),
 (' ', '<O>'),
 ('10.11.10.1', 'B-IP'),
 (' ', '<O>'),
 ('"', '<O>'),
 ('GET', '<O>'),
 (' ', '<O>'),
 ('/v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail', 'B-PATH'),
 (' ', '<O>'),
 ('HTTP/1.1', '<O>'),
 ('"', '<O>'),
 (' ', '<O>'),
 ('status', '<O>'),
 (':', '<O>'),
 (' ', '<O>'),
 ('200', '<O>'),
 (' ', '<O>'),
 ('len', '<O>'),
 (':', '<O>'),
 (' ', '<O>'),
 ('1892', '<O>'),
 (' ', '<O>'),
 ('time', '<O>'),
 (':', '<O>'),
 (' ', '<O>'),
 ('0.2479329', '<O>')]

In [17]:
with open('./header_check.txt','w') as f:
    for tokens,tags in zip(header_tokens,header_anno):
        for token,tag in zip(tokens,tags):
            f.write(token+' '+tag+'\n')
        f.write('\n')

In [18]:
with open('./msg_check.txt','w') as f:
    for tokens,tags in zip(msg_tokens,msg_anno):
        for token,tag in zip(tokens,tags):
            f.write(token+' '+tag+'\n')
        f.write('\n')

In [19]:
df=pd.DataFrame({
    'tokens':[h+m for h,m in zip(header_tokens,msg_tokens)],
    'tags':[h+m for h,m in zip(header_anno,msg_anno)],
    'raw_log':[h+m for h,m in zip(header,msg)]
    })
df.to_csv('./openstack_ds.csv')